In [121]:
import os
import numpy as np
import pandas as pd
import re
import sys  
import matplotlib.pyplot as plt

In [122]:
text = pd.read_csv("./datasets/hongloumeng.csv")
import jieba
import jieba.analyse

vorc = [jieba.analyse.extract_tags(i, topK=1000) for i in text["text"]]
vorc = [" ".join(i) for i in vorc]

In [123]:
from  sklearn.feature_extraction.text import CountVectorizer
vertorizer = CountVectorizer(max_features=5000)
train_data_features = vertorizer.fit_transform(vorc)

In [124]:
train_data_features = train_data_features.toarray()

In [125]:
train_data_features.shape

(120, 5000)

In [126]:
labels = np.array([[0] * 80 + [1] * 40]).reshape(-1 ,1) # 目标值
labels.shape

(120, 1)

In [127]:
# 分层抽样
from sklearn.model_selection import train_test_split
# train_data_features = train_data_features[0:80]
X_train, X_test, Y_train, Y_test = train_test_split(train_data_features, labels, 
                                                    test_size = 0.2, stratify=labels)

In [128]:
Y_test.mean()

0.3333333333333333

In [129]:
# 分别取前80回中的60回，后40回中的30回进行训练

In [130]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

In [131]:
gnb = GaussianNB()
gnb.fit(X_train, Y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None)

In [132]:
# 进行预测
y_pred = gnb.predict(X_test)

In [133]:
from sklearn.metrics import accuracy_score

In [134]:
accuracy_score(Y_test, y_pred)

0.8333333333333334

In [135]:
from sklearn.model_selection import StratifiedShuffleSplit

In [136]:
sss = StratifiedShuffleSplit(n_splits=10,test_size=0.2)#分成5组，测试比例为0.25，训练比例是0.75

In [168]:
scores = []
for train_index, test_index in sss.split(train_data_features, labels):
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    Y_train, Y_test = labels[train_index], labels[test_index]
    gnb = GaussianNB()
    gnb.fit(X_train, Y_train)
    Y_pred = gnb.predict(X_test)
    scores.append(accuracy_score(Y_test, Y_pred))
print(scores)
print(np.array(scores).mean())

[0.9166666666666666, 0.8333333333333334, 1.0, 0.875, 0.75, 0.8333333333333334, 0.8333333333333334, 0.9583333333333334, 0.875, 0.8333333333333334]
0.8708333333333333


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## 验证

In [175]:
labels_val = np.array([[0] * 40 + [1] * 40]).reshape(-1 ,1) # 目标值
sss_val = StratifiedShuffleSplit(n_splits=5,test_size=0.2)#分成5组，测试比例为0.25，训练比例是0.75
scores = []
train_data_features_val = train_data_features[0:80]
for train_index, test_index in sss_val.split(train_data_features_val, labels_val):
    X_train, X_test = train_data_features_val[train_index], train_data_features_val[test_index]
    Y_train, Y_test = labels_val[train_index], labels_val[test_index]
    gnb = GaussianNB()
    gnb.fit(X_train, Y_train)
    Y_pred = gnb.predict(X_test)
    scores.append(accuracy_score(Y_test, Y_pred))
print(scores)
print(np.array(scores).mean())

[0.8125, 0.875, 0.75, 0.875, 0.75]
0.8125


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
